In [ ]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

In [5]:
 !pip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [10]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-16,"$1,665.41","$1,727.29","$1,653.17","$1,716.67","$1,716.67",6281023479
2023-06-17,"$1,716.67","$1,766.76","$1,714.15","$1,727.20","$1,727.20",4875187477
2023-06-18,"$1,727.19","$1,746.51","$1,718.11","$1,720.58","$1,720.58",3820038842
2023-06-19,"$1,720.51","$1,745.13","$1,705.81","$1,737.66","$1,737.66",5210480121
2023-06-20,"$1,736.88","$1,793.31","$1,715.34","$1,792.12","$1,792.12",7171072578


In [11]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2050 entries, 2017-11-09 to 2023-06-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2050 non-null   float64
 1   High       2050 non-null   float64
 2   Low        2050 non-null   float64
 3   Close      2050 non-null   float64
 4   Adj Close  2050 non-null   float64
 5   Volume     2050 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 112.1 KB


In [12]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [13]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [14]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [15]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [16]:
df.tail()

,ds,y
2045,2023-06-16,"$1,665.41"
2046,2023-06-17,"$1,716.67"
2047,2023-06-18,"$1,727.19"
2048,2023-06-19,"$1,720.51"
2049,2023-06-20,"$1,736.88"


In [17]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [18]:
m = Prophet(
    seasonality_mode="multiplicative"
)

m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptzi0ed2j/v3d4mnea.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptzi0ed2j/c9u5x6b0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94381', 'data', 'file=/tmp/tmptzi0ed2j/v3d4mnea.json', 'init=/tmp/tmptzi0ed2j/c9u5x6b0.json', 'output', 'file=/tmp/tmptzi0ed2j/prophet_modelrhiplysw/prophet_model-20230621095415.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:54:15 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:54:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [19]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2410,2024-06-15
2411,2024-06-16
2412,2024-06-17
2413,2024-06-18
2414,2024-06-19


In [20]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2410,2024-06-15,$914.01,$-783.37,"$2,362.75"
2411,2024-06-16,$903.48,$-758.04,"$2,394.28"
2412,2024-06-17,$891.29,$-785.43,"$2,295.99"
2413,2024-06-18,$875.33,$-700.04,"$2,329.91"
2414,2024-06-19,$860.67,$-742.80,"$2,270.64"


In [21]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

1138.9942597788931

In [22]:
plot_plotly(m, forecast)

In [23]:
plot_components_plotly(m, forecast)